# Points of Dispense - Global
#### Minimize total distance and vary number of PODs
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Optimization

Read in data from files

In [4]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')

In [5]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [6]:
dist = dist_miles.values
N = population['population'].values
prop = population['global'].values

Create indices for block groups and PODs

In [7]:
blocks = range(len(N))
pods = range(len(dist[0]))

Vary number of PODs to open

In [8]:
pods_open = range(9,48,2)
pods_open_results = {}

In [9]:
for i in pods_open: 
    ### Initialize model for POD locations
    m = gp.Model('POD_locations')
    
    ### Add decision variables
    x = m.addVars(pods, vtype=GRB.BINARY, name='x')
    y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='y')
    
    ### Set objective to minimize total distance across the population
    obj = gp.quicksum(dist[j,i] * x[i] * y[j, i] * N[j] * prop[j] 
                  for j in blocks for i in pods)
    m.setObjective(obj, GRB.MINIMIZE)
    
    ### Add constraints
    #  y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened
    m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')
    
    # each block group must be assigned one shelter
    m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')
    
    # number of PODs opened must be less than num_pods
    m.addConstr((gp.quicksum(x[i] for i in pods) <= i), name='pods_opened')
    
    ### Optimize
    print('Solving for:', i, 'PODs')
    m.optimize()
    
    ### Analysis
    block_pod_list = [[j,i] for j in blocks for i in pods if y[j, i].x==1]
    pods_open_results[i] = block_pod_list

Using license file C:\Users\Elizabeth\gurobi.lic
Academic license - for non-commercial use only
Solving for: 9 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 52801 rows, 51747 columns and 155147 nonzeros
Model fingerprint: 0x19ddc650
Model has 51371 quadratic objective terms
Variable types: 0 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve time: 0.20s
Presolved: 104172 rows, 103118 columns, 309260 nonzeros
Variable types: 0 continuous, 103118 integer (103118 binary)
Found heuristic solution: objective 9080840.5175

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 5.232789e+06, 6711 iterations, 1.27 seconds

    Nodes    |    Current Node 

In [10]:
pods_open_total2 = open('pods_open_total2','wb') 
pickle.dump(pods_open_results, pods_open_total2)                      
pods_open_total2.close()